In [1]:
%load_ext nb_black
from functions_questions import add_to_file
from functions_variable_overview import *

<IPython.core.display.Javascript object>

## Create Table 

Read in description table 

In [2]:
table = pd.read_csv("tables/covid_variable_description.csv", sep=";")

<IPython.core.display.Javascript object>

Rename columns so they are easier to handle for the functions

In [3]:
table = table.rename(
    columns={
        "L_CoronavirusImpact_wave3_4p.dta": "wave#3",
        "L_CoronavirusImpact_4.0p.dta": "wave#2",
        "L_CoronavirusImpact_3p.dta": "wave#1",
        "L_CoronavirusImpact_wave4_4p.dta": "wave#4",
        "new_name": "variable",
        "topic_english": "Topic",
        "type": "Type",
        "new_name": "Variable",
    }
)

<IPython.core.display.Javascript object>

Select wave and language indicator. 
- `language` should be one lower-case letter. 
- `waves` should be a list of one or more values.

In [20]:
waves = [1, 2]
language_short = "e"
language_long = "english"

<IPython.core.display.Javascript object>

Create decription table and save to csv. If the table only includes one wave, it is saved to the respective language folder. If it is created for multiple waves, it is saved to "/docs/source/".

In [21]:
if len(waves) == 1:
    variable_table = create_description_table(table, waves, language_short)
    path = f"../docs/source/wave-{waves[0]}/"
else:
    variable_table = create_description_table(
        table, waves, language_short, routing=False
    )
    # Update path for multiple-wave overview:
    path = f"../docs/source/"

<IPython.core.display.Javascript object>

In [22]:
if language_short == "e":
    variable_table.to_csv(
        f"{path}variable_table-waves-{'-'.join(str(i) for i in waves)}-{language_short}.csv",
        sep=",",
        index=False,
    )

<IPython.core.display.Javascript object>

## Create index file for questions sorted according to topics

In [13]:
if len(waves) == 1:
    topic_table = create_description_table(
        table, waves, language_short, return_links_as_symbols=False, routing=False
    )
    topic_dict = topic_table.groupby("Topic")[f"Links Wave {waves[0]}"].unique()
    for key in ["skip", "Skip"]:
        if key in topic_dict.keys():
            topic_dict = topic_dict.drop(key)
    create_overview_page(topic_dict, waves[0], language_long, path)

<IPython.core.display.Javascript object>